## СЕНЗОРСКИ СИСТЕМИ

*Лабораториска вежба бр. 1*

### 2Д трилатерација

Изработка на програма за локализација на јазли кај безжична сензорска мрежа (во 2Д), со употреба на техниката на трилатерација.

Влезни аргументи на програмата:

* N - број на јазли во мрежата

* L – должина на областа каде е дистрибуирана мрежата

* R - радио опсег

* r – зашумување на сигналот

* f – фракција (процент) на anchor јазли

In [1]:
import random
import math
from math import cos, sin, pi
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from matplotlib.pyplot import *
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

# Трилатерација

*Искористениот код за алгоритмот е преземен од https://github.com/rkcosmos/Location_Trilateration/blob/master/map.ipynb*

In [2]:
def distance(p1,p2):
    dx = p1[0]-p2[0]
    dy = p1[1]-p2[1]
    distance = np.sqrt(dx*dx+dy*dy)
    return distance

def circle_intersection(circle1, circle2):
    '''
    Find 2 intersection points from 2 circles.
    if circles are non overlap, return middle point between 2 perimeters
    '''  
    x1,y1,r1 = circle1
    x2,y2,r2 = circle2
    dx,dy = x2-x1,y2-y1
    d = np.sqrt(dx*dx+dy*dy)
    #d = distance([x1,y1],[x2,y2])
    
    # non-overlapping circles
    if d >= r1+r2:
        return (( (d+r1-r2)/(2*d)*(x2-x1)+x1, (d+r1-r2)/(2*d)*(y2-y1)+y1 ),) 

    # one circle inside another
    elif r1 > (d+r2) or r2 > (d+r1):
        if dx == 0: angle = pi/2
        else: angle = np.arctan(dy/dx)
        p11 = [ x1+r1*cos(angle), y1+r1*sin(angle) ]
        p12 = [ x1-r1*cos(angle), y1-r1*sin(angle) ]
        p21 = [ x2+r2*cos(angle), y2+r2*sin(angle) ]
        p22 = [ x2-r2*cos(angle), y2-r2*sin(angle) ]
        
        point_set = [[p11,p21],[p11,p22],[p12,p21],[p12,p22]]
        
        dist_set = [distance(item[0],item[1]) for item in point_set]
        points = point_set[np.argmin(dist_set)]
        points = np.array(points)
        points = sum(points)/len(points)

        return (points,)
    
    # overlapping circles
    else:
        a = (r1*r1-r2*r2+d*d)/(2*d)
        h = np.sqrt(r1*r1-a*a)
        xm = x1 + a*dx/d
        ym = y1 + a*dy/d
        xs1 = xm + h*dy/d
        xs2 = xm - h*dy/d
        ys1 = ym - h*dx/d
        ys2 = ym + h*dx/d

        return (xs1,ys1),(xs2,ys2)

def plot_t(cs):
   
    cmap = cm.plasma
    
    fig, ax = plt.subplots() 
    ax.set(xlim=[-10, 10], ylim=[-10, 10], aspect=1)
    
    for i,c in enumerate(cs):
        ax.add_artist(plt.Circle(c[:2], c[2], color=cmap(i/len(cs)), alpha=0.5))
        ax.plot(c[0], c[1], 'or')

    return fig, ax

def choose_point(points, circle, order):
    '''
    1. choose point nearest to perimeter of given circle
    2. between choosen point and perimeter of circle, calculate location from weight average 
    
    Args:
        points: [[x1,y1],[x2,x2],...]
        circle: [center_x, center_y, radius]
        order = number of circle in this calculation
    Returns:
        result: [x,y]
    ''' 
    distance = ()
    for point in points:
        dx = point[0]-circle[0]
        dy = point[1]-circle[1]
        dist = np.sqrt(dx*dx+dy*dy) - circle[2] 
        distance = distance + (dist,)
    
    arg = np.argmin([abs(i) for i in distance])
    choosen_point = points[arg]
    # ref_point = point on c3 perimeter closest to choosen_point
    ref_point = [circle[0]+(choosen_point[0]-circle[0])*circle[2]/(circle[2]+distance[arg]), 
                 circle[1]+(choosen_point[1]-circle[1])*circle[2]/(circle[2]+distance[arg])]
    result = [((order-1)*choosen_point[0]+ref_point[0])/order, ((order-1)*choosen_point[1]+ref_point[1])/order]
        
    return result


def trilateration(c1, c2, c3):
    
    points = circle_intersection(c1, c2)
    result = choose_point(points, c3, 3)
    
    #fig, ax = plot_t([c1, c2, c3])
    #ax.plot(result[0], result[1], "*")
    return result

In [3]:
class Node:
    """
        
    """

    def __init__(self):
        self.id = None
        self.x_coord = None
        self.y_coord = None
        self.isAnchor = 0
        self.unreliability = math.inf
        self.x_estimated = None
        self.y_estimated = None
        self.neighbours = []
        self.distances = []
        self.estimated = False

    def euclideanDistance(self, node):
        x1 = self.x_coord
        y1 = self.y_coord
        x2 = node.x_coord
        y2 = node.y_coord
        result = math.sqrt(math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2))
        return result
    
    def minUnreliability(self, node):
        if self.unreliability <= node.unreliability:
            return -1
        else:
            return 0

    def getAnchorNeighbours(self):
        anchors = []
        for neighbour in self.neighbours:
            if neighbour.isAnchor > 0:
                anchors.append(neighbour)
        return anchors
    
    def getErrorOfEstimation(self):
        x_diff = math.pow(self.x_estimated - self.x_coord, 2)
        y_diff = math.pow(self.y_estimated - self.y_coord, 2)
        error = math.sqrt( x_diff + y_diff)
        return error

    def estimateLocation(self, cycles, heuristic=1, r=10):
        if self.estimated == True:
            return False
        if heuristic == 1:
            #print("Heuristic 1")
            return self.estimateLocation1(cycles, r)
        elif heuristic == 2:
            #print("Heuristic 2")
            return self.estimateLocation2(cycles, r)
        else:
            pass
            
    def __str__(self):
        return "id = " + str(self.id) + "\n" + \
               "\tx = " + str(self.x_coord) + ", y = " + str(self.y_coord) + "\n" + \
               "\tisAnchor = " + str(self.isAnchor) + "\n" + \
               "\tunreliability = " + str(self.unreliability) + "\n" + \
               "\tx_estimated = " + str(self.x_estimated) + ", y = " + str(self.y_estimated) + "\n" + \
               "\tNeighbours: " + str([n.id for n in self.neighbours]) + "\n"

    
    def estimateLocation1(self, cycles, r):
        self.neighbours.sort(key=lambda neighbour: self.euclideanDistance(neighbour))
        anchors = self.getAnchorNeighbours()
        if len(anchors) < 3:
            return False
        else:
            neighbour1 = anchors[0]
            neighbour2 = anchors[1]
            neighbour3 = anchors[2]

            rand = np.random.randint(0, 10)  # to simulate the incorrect distance measured
            noise = None
            if (rand < 5):
                noise = 1 - r / 100.0
            else:
                noise = 1 + r / 100.0

            r1 = self.euclideanDistance(neighbour1) * noise
            r2 = self.euclideanDistance(neighbour2) * noise
            r3 = self.euclideanDistance(neighbour3) * noise

            pt_a = [neighbour1.x_coord, neighbour1.y_coord, r1]
            pt_b = [neighbour2.x_coord, neighbour2.y_coord, r2]
            pt_c = [neighbour3.x_coord, neighbour3.y_coord, r3]

            result = trilateration(pt_a, pt_b, pt_c)

            self.estimated = True
            self.x_estimated = result[0]
            self.y_estimated = result[1]
            self.isAnchor = cycles + 1
            self.unreliability = neighbour1.unreliability + neighbour2.unreliability + neighbour3.unreliability + 1
            return True
    
    def estimateLocation2(self, cycles, r):
        self.neighbours.sort(key=lambda neighbour: self.minUnreliability(neighbour))
        anchors = self.getAnchorNeighbours()
        if len(anchors) < 3:
            return False
        else:
            neighbour1 = anchors[0]
            neighbour2 = anchors[1]
            neighbour3 = anchors[2]

            rand = np.random.randint(0, 10)  # to simulate the incorrect distance measured
            noise = None
            if (rand < 5):
                noise = 1 - r / 100.0
            else:
                noise = 1 + r / 100.0

            r1 = self.euclideanDistance(neighbour1) * noise
            r2 = self.euclideanDistance(neighbour2) * noise
            r3 = self.euclideanDistance(neighbour3) * noise

            pt_a = [neighbour1.x_coord, neighbour1.y_coord, r1]
            pt_b = [neighbour2.x_coord, neighbour2.y_coord, r2]
            pt_c = [neighbour3.x_coord, neighbour3.y_coord, r3]

            result = trilateration(pt_a, pt_b, pt_c)

            self.estimated = True
            self.x_estimated = result[0]
            self.y_estimated = result[1]
            self.isAnchor = cycles + 1
            self.unreliability = neighbour1.unreliability + neighbour2.unreliability + neighbour3.unreliability + 1
            return True
    
    
def generateNodes(N, L, R, r, f):
    nodes = dict()
    coordinates = [(float(x) * L, float(y) * L) for x, y in np.random.rand(N, 2)]
    # print(coordinates)
    anchorNodesNumber = int(f / 100 * N)
    for id in range(N):
        node = Node()
        node.id = id
        x = coordinates[id][0]
        y = coordinates[id][1]
        node.x_coord = x
        node.y_coord = y
        if id < anchorNodesNumber:
            node.isAnchor = 1
            node.unreliability = 0
            node.x_estimated = node.x_coord
            node.y_estimated = node.y_coord
            node.estimated = True
        nodes[id] = node
    return nodes


def plot(nodes, N, L, f):
    anchorNodesNumber = int(f / 100 * N)
    x_anchorNodes = [nodes[i].x_coord for i in range(anchorNodesNumber)]
    y_anchorNodes = [nodes[i].y_coord for i in range(anchorNodesNumber)]

    x_other = [nodes[i].x_coord for i in range(anchorNodesNumber + 1, len(nodes))]
    y_other = [nodes[i].y_coord for i in range(anchorNodesNumber + 1, len(nodes))]

    anchorNodes = {"x": x_anchorNodes,
                   "y": y_anchorNodes,
                   "mode": "markers",
                   "marker": {"color": "red", "size": 8},
                   "name": "Anchor Nodes"
                   }
    otherNodes = {"x": x_other,
                  "y": y_other,
                  "mode": "markers",
                  "marker": {"color": "blue", "size": 5},
                  "name": "Other Nodes"
                  }
    data = [anchorNodes, otherNodes]

    iplot(data)


def findNeighbours(nodes, R):
    for id in range(len(nodes.keys())):
        neighbours = []
        distances = []
        for id2 in range(len(nodes.keys())):
            if (id == id2):
                continue
            distance = nodes[id].euclideanDistance(nodes[id2])
            if (distance < R):
                neighbours.append(nodes[id2])
                distances.append(distance)
        nodes[id].neighbours = neighbours
        nodes[id].distances = distances
    return nodes



"""
    generateNodes(N, L, R, r, f=20)
    plot(nodes, N, L, f)
    
"""
nodes = generateNodes(100, 100, 0, 0, 20)
plot(nodes, 100, 100, 20)


"""
    findNeighbours(nodes, R=20)
"""
nodes = findNeighbours(nodes, 20)

In [4]:
def estimateLocations(nodes, heuristic, r = 10, iterative):
    global cycles 
    cycles = 0
    if iterative:
        while (True):
            cycles = cycles + 1
            estimatedSmth = False
            for idx in range(len(nodes)):
                # print(nodes[idx].estimateLocation())
                est = nodes[idx].estimateLocation(cycles, heuristic, r)
                if est:
                    # print(nodes[idx])
                    estimatedSmth = True
                    nodes[idx].estimated = True
            if not estimatedSmth:
                break
    else:
        cycles = cycles + 1
        for idx in range(len(nodes)):
            est = nodes[idx].estimateLocation(cycles, heuristic, r)
            
    return nodes

nodes = estimateLocations(nodes, 1)
            
black_nodes = []
anchor_nodes = []
estimated_nodes = []


for idx in range(len(nodes)):
    if nodes[idx].isAnchor == 1:
        anchor_nodes.append(nodes[idx])
    elif nodes[idx].estimated:
        estimated_nodes.append(nodes[idx])
    else:
        black_nodes.append(nodes[idx])

print("Total of Black Nodes:", len(black_nodes))
print("Total of Anchor Nodes:", len(anchor_nodes))
print("Total of Estimated Nodes:", len(estimated_nodes))
#print("Cycles needed to estimate as much as possible nodes: ", cycles)

Total of Black Nodes: 34
Total of Anchor Nodes: 20
Total of Estimated Nodes: 46


In [5]:
"""
    This is a function that plots the Anchor Nodes (RED), Nodes that could have been estimated (YELLOW)
    and nodes that couldn't have been estimated (BLACK)

"""

def plot2(black_nodes, anchor_nodes, estimated_nodes, cycles):
    x_black_nodes = [black_nodes[i].x_coord for i in range(len(black_nodes))]
    y_black_nodes = [black_nodes[i].y_coord for i in range(len(black_nodes))]

    x_anchor_nodes = [anchor_nodes[i].x_coord for i in range(len(anchor_nodes))]
    y_anchor_nodes = [anchor_nodes[i].y_coord for i in range(len(anchor_nodes))]
    
    x_estimated_nodes = [estimated_nodes[i].x_coord for i in range(len(estimated_nodes))]
    y_estimated_nodes = [estimated_nodes[i].y_coord for i in range(len(estimated_nodes))]
    
    name_black = "Black Nodes (" + str(len(black_nodes)) + ")"
    black_nodes = {"x" : x_black_nodes,
                   "y" : y_black_nodes,
                   "mode" : "markers",
                   "marker" : {"color" : "black", "size":5},
                   "name" : name_black
                  }
    
    name_anchor = "Anchor Nodes (" + str(len(anchor_nodes)) + ")"
    anchor_nodes = {"x" : x_anchor_nodes,
                  "y" : y_anchor_nodes,
                  "mode" : "markers",
                  "marker" : {"color" : "red", "size" : 8},
                  "name" : name_anchor
                 }
    
    name_estimated = "Estimated Nodes (" + str(len(estimated_nodes)) + ")"
    estimated_nodes = {"x" : x_estimated_nodes,
                  "y" : y_estimated_nodes,
                  "mode" : "markers",
                  "marker" : {"color" : "yellow", "size" : 5},
                  "name" : name_estimated
                 }
    
    data = [black_nodes, anchor_nodes, estimated_nodes]

    iplot(data)
    

    
"""
    This is a function that plots the correct location of a node (red) and the estimated location of the
    node by trilateration (blue)
"""
def plot3(original_node, estimated_node):
    x_original_node = [original_node[i][0] for i in range(len(original_node))]
    y_original_node = [original_node[i][1] for i in range(len(original_node))]

    x_estimated_node = [estimated_node[i][0] for i in range(len(estimated_node))]
    y_estimated_node = [estimated_node[i][1] for i in range(len(estimated_node))]
    
    original_node_name = "Correct Location"
    original_nodes = {"x" : x_original_node,
                   "y" : y_original_node,
                   "mode" : "markers",
                   "marker" : {"color" : "red", "size":5},
                   "name" : original_node_name
                  }
    
    estimated_node_name = "Estimated Node"
    estimated_nodes = {"x" : x_estimated_node,
                  "y" : y_estimated_node,
                  "mode" : "markers",
                  "marker" : {"color" : "blue", "size" : 5},
                  "name" : estimated_node_name
                 }
    
    
    data = [original_nodes, estimated_nodes]

    iplot(data)
    

original_node = []
estimated_node = []


for node in estimated_nodes:
    node_original = node.x_coord, node.y_coord
    node_estimated = node.x_estimated, node.y_estimated
    
    original_node.append(node_original)
    estimated_node.append(node_estimated)
    

# for id in range(len(original_node)):
#     print(original_node[id])
#     print(estimated_node[id])
#     print()

x_original_node = [original_node[i][0] for i in range(len(original_node))]
y_original_node = [original_node[i][1] for i in range(len(original_node))]

x_estimated_node = [estimated_node[i][0] for i in range(len(estimated_node))]
y_estimated_node = [estimated_node[i][1] for i in range(len(estimated_node))]

In [6]:
plot2(black_nodes, anchor_nodes, estimated_nodes, cycles)

In [7]:
plot3(original_node, estimated_node)

In [8]:
data = []

original_node_name = "Correct Location"
original_nodes = {"x" : x_original_node,
               "y" : y_original_node,
               "mode" : "markers",
               "marker" : {"color" : "red", "size":5},
               "name" : original_node_name
              }

estimated_node_name = "Estimated Nodes"
estimated_nodes1 = {"x" : x_estimated_node,
              "y" : y_estimated_node,
              "mode" : "markers",
              "marker" : {"color" : "blue", "size" : 5},
              "name" : estimated_node_name
             }    

x_black_nodes = [black_nodes[i].x_coord for i in range(len(black_nodes))]
y_black_nodes = [black_nodes[i].y_coord for i in range(len(black_nodes))]

name_black = "Black Nodes (" + str(len(black_nodes)) + ")"
black_nodes = {"x" : x_black_nodes,
               "y" : y_black_nodes,
               "mode" : "markers",
               "marker" : {"color" : "black", "size":5},
               "name" : name_black
              }

data.append(original_nodes)
data.append(estimated_nodes1)
data.append(black_nodes)

for node in estimated_nodes:
    trace = go.Scatter(
        x = [node.x_coord, node.x_estimated],
        y = [node.y_coord, node.y_estimated],
        mode = 'lines',
        name = 'Node ' + str(node.id),
        marker = dict(
            size = 10,
            color = 'black',
            line = dict(
                width = 1,
                color = 'black'
            )
        )
    )
    data.append(trace)

In [9]:
iplot(data)

In [10]:
"""
    generateNodes(N, L, R, r, f=20)
"""
nodes = generateNodes(100, 100, 0, 0, 20)

"""
    findNeighbours(nodes, R=25)
"""
nodes = findNeighbours(nodes, 25)

ErrorHeuristic1 = 0
ErrorHeuristic2 = 0

In [11]:
def calculateError(nodes, N, f):
    estimated = 0
    error = 0
    for idx in range(len(nodes)):
        if nodes[idx].isAnchor > 1:
            estimated = estimated + 1
            error = error + nodes[idx].getErrorOfEstimation()
    return error/(N-f)

# 1. Неитеративен пристап
-------------------------------------------------------------------------------------------------------------------------------
## 1.1. Процент на локализирани јазли во зависност од R и f


In [12]:
N = 100
L = 100
R = 20
r = 10
# f = {5,10,15,20,25,30,35,40}
heuristic = 1
errorsH1 = []

nodes = None
localized_nodes_percentage1 = []
for f in range(5, 41, 5):
    error = 0
    #for i in range(0, 15):
    nodes = generateNodes(N, L, R, r, f)
    nodes = findNeighbours(nodes, R)
    nodes = estimateLocations(nodes, heuristic, iterative = False)

    total = 0
    for index in nodes:
        if nodes[index].isAnchor > 1:
            total += 1;
    localized_nodes_percentage1.append(total*100/(N-f))

print(localized_nodes_percentage1)

[0.0, 12.222222222222221, 31.764705882352942, 67.5, 65.33333333333333, 77.14285714285714, 83.07692307692308, 85.0]


In [13]:
N = 100
L = 100
R = 40
r = 10
# f = {5,10,15,20,25,30,35,40}
heuristic = 1
errorsH1 = []

nodes = None
localized_nodes_percentage2 = []
for f in range(5, 41, 5):
    error = 0
    #for i in range(0, 15):
    nodes = generateNodes(N, L, R, r, f)
    nodes = findNeighbours(nodes, R)
    nodes = estimateLocations(nodes, heuristic, iterative = False)

    total = 0
    for index in nodes:
        if nodes[index].isAnchor > 1:
            total += 1;
    localized_nodes_percentage2.append(total*100/(N-f))

print(localized_nodes_percentage2)

[91.57894736842105, 98.88888888888889, 98.82352941176471, 98.75, 100.0, 100.0, 100.0, 100.0]


In [14]:
r = [5, 10, 15, 20, 25, 30, 35, 40]

trace0 = go.Scatter(
    x = r,
    y = localized_nodes_percentage1,
    mode = 'lines',
    name = 'R = 20'
)

trace1 = go.Scatter(
    x = r,
    y = localized_nodes_percentage2,
    mode = 'lines',
    name = 'R = 40'
)

data = [trace0, trace1]

iplot(data)

 ## Процент на локализирани јазли во зависност од R и r 

In [15]:
N = 100
L = 100
R = 30
# r = {5,10,15,20,25,30,35,40} 
f = 20
heuristic = 1
errorsH1 = []

nodes = None
localized_nodes_percentage1 = []
for r in range(5, 41, 5):
    # error = 0
    #for i in range(0, 15):
    nodes = generateNodes(N, L, R, r, f)
    nodes = findNeighbours(nodes, R)
    nodes = estimateLocations(nodes, heuristic, iterative = False)

    total = 0
    for index in nodes:
        if nodes[index].isAnchor > 1:
            total += 1;
    localized_nodes_percentage1.append(total*100/(N-f))

print(localized_nodes_percentage1)

[93.75, 100.0, 95.0, 96.25, 96.25, 96.25, 93.75, 96.25]


In [16]:
N = 100
L = 100
R = 40
# r = {5,10,15,20,25,30,35,40} 
f = 20
heuristic = 1
errorsH1 = []

nodes = None
localized_nodes_percentage1 = []
for r in range(5, 41, 5):
    # error = 0
    #for i in range(0, 15):
    nodes = generateNodes(N, L, R, r, f)
    nodes = findNeighbours(nodes, R)
    nodes = estimateLocations(nodes, heuristic, iterative = False)

    total = 0
    for index in nodes:
        if nodes[index].isAnchor > 1:
            total += 1;
    localized_nodes_percentage1.append(total*100/(N-f))

print(localized_nodes_percentage1)

[97.5, 100.0, 100.0, 97.5, 100.0, 97.5, 98.75, 100.0]


In [17]:
r = [5, 10, 15, 20, 25, 30, 35, 40]

trace0 = go.Scatter(
    x = r,
    y = localized_nodes_percentage1,
    mode = 'lines',
    name = 'R = 30'
)

trace1 = go.Scatter(
    x = r,
    y = localized_nodes_percentage2,
    mode = 'lines',
    name = 'R = 40'
)

data = [trace0, trace1]

iplot(data)

 ## 1.2. Анализа на просечната грешка на локализација во однос на различни параметри
 
 ### Просечната грешка на локализација vs фракцијата на anchor јазли

In [18]:
N = 100
L = 100
R = 20
r = 10
# f = {10,20,30,40,50}
heuristic = 1
errorsH1 = []

nodes = None
for f in range(10, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, iterative = False)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[0.3950106149126328, 1.196290325044397, 1.4316791163432403, 1.4967170264892957, 1.5908924621782992]


In [19]:
f = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = f,
    y = errorsH1,
    mode = 'lines',
)


data = [trace0]

iplot(data)

 ### Просечната грешка на локализација во однос на радио опсегот

In [20]:
N = 100
L = 100
f = 20
r = 10
#R = {10, 20, 30, 40, 50}

heuristic = 1
errorsH1 = []

nodes = None
for R in range(10, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, iterative = False)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[0.03276211098389042, 1.0345386721481145, 2.0161248018263884, 2.047795770677864, 1.9184590086104956]


In [21]:
R = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = R,
    y = errorsH1,
    mode = 'lines',
)


data = [trace0]

iplot(data)

 ### Просечната грешка на локализација vs radio range error

In [22]:
N = 100
L = 100
f = 20
R = 20
# r = {10...50}

heuristic = 1
errorsH1 = []

nodes = None
for r in range(10, 51, 5):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, r)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[1.2077296181088275, 1.7143778995113554, 2.0792236194968963, 2.4723988642073587, 2.7794950163311656, 2.958675256847208, 3.317776855772415, 3.8320458569918494, 4.107085846894074]


In [23]:
r = [10, 15, 20, 25, 30, 35, 40, 45, 50]

trace0 = go.Scatter(
    x = r,
    y = errorsH1,
    mode = 'lines',
)


data = [trace0]

iplot(data)

# 2. ИТЕРАТИВЕН ПРИСТАП 
---------------------------------------------------------------------------------------------------
Искористени две евристики: 
1) H1 = најблиските anchor јазли <br>
2) H2 = најрелевантните anchor јазли


### Анализа на грешката на локализација во однос на фракцијата на anchor јазли (plot 1)

In [39]:
N = 100
L = 100
R = 30
r = 10
# f = {10,20,30,40,50}
heuristic = 1
errorsH1 = []

nodes = None
for f in range(10, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, iterative = True)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[2.1495805909295824, 1.9518280249798963, 1.7951291557381321, 1.8770809912238657, 1.7624960944848538]


In [40]:
heuristic = 2
errorsH2 = []

nodes = None
for f in range(10, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, iterative = True)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/15)

print(errorsH2)

[3.7866633759089137, 3.713759710242129, 3.8817058293423314, 3.830347933497421, 3.7259482977967995]


In [42]:
f = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = f,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = f,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)

### Анализа на просечната грешка на локализација и радио опсегот R (plot 2)

In [27]:
N = 100
L = 100
f = 25
r = 10
#R = {20, 30, 40, 50}

heuristic = 1
errorsH1 = []

nodes = None
for R in range(20, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[1.2877925055271708, 1.8095611929664563, 2.014499230098814, 2.1238787189784207]


In [28]:
heuristic = 2
errorsH2 = []

nodes = None
for R in range(20, 51, 10):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/15)

print(errorsH2)

[2.0318392018363087, 3.772865308777385, 4.682252754277485, 5.771319747224581]


In [29]:
R = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = R,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = R,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)

### Анализа на radio range error и просечната грешка на локализација (plot 3)

In [43]:
N = 100
L = 100
f = 25
R = 35
# r = {10...50}

heuristic = 1
errorsH1 = []

nodes = None
for r in range(10, 51, 5):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, r, iterative = True)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/15)

print(errorsH1)

[1.9084671256982395, 2.6963638114781547, 3.4197950698334747, 4.162032622546215, 4.71142277627101, 5.1673514942098215, 5.6514521036410965, 6.090506593306598, 6.601613971393794]


In [44]:
heuristic = 2
errorsH2 = []

nodes = None
for r in range(10, 51, 5):
    error = 0
    for i in range(0, 15):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, r, iterative = True)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/15)

print(errorsH2)

[4.507181091769395, 6.1068839432942665, 7.683427805759647, 8.67213332730041, 9.94295885033251, 11.89639754073273, 12.585938746687566, 14.367122085369239, 15.036883976786912]


In [46]:
r = [10, 15, 20, 25, 30, 35, 40, 45, 50]

trace0 = go.Scatter(
    x = r,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = r,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)